# Classifiers and training

In [63]:
import time
import scipy.io
import mne
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import antropy as ant
import EntropyHub as EH
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

Load preprocessed matrix data of normal and fatigue states for training.

TODO what subsets did i make?

In [17]:
AE_normal_matrix = np.load("data/AE_normal_matrix.npy")
PE_normal_matrix = np.load("data/PE_normal_matrix.npy")
SE_normal_matrix = np.load("data/SE_normal_matrix.npy")
FE_normal_matrix = np.load("data/FE_normal_matrix.npy")

AE_fatigue_matrix = np.load("data/AE_fatigue_matrix.npy")
PE_fatigue_matrix = np.load("data/PE_fatigue_matrix.npy")
SE_fatigue_matrix = np.load("data/SE_fatigue_matrix.npy")
FE_fatigue_matrix = np.load("data/FE_fatigue_matrix.npy")

In [39]:
NUM_PEOPLE = 12
NUM_EPOCHS = 300
NUM_CHANNELS = 30

X_ALL = {
    "AE" : [],
    "PE" : [],
    "SE" : [],
    "FE" : []
}

"""
y_ALL = {
    "AE" : [],
    "PE" : [],
    "SE" : [],
    "FE" : []
}
"""

# TODO remove
def create_y():
    normal = [0 for i in range(NUM_EPOCHS)]
    fatigue = [1 for i in range(NUM_EPOCHS)]
    return normal + fatigue

for person in range(NUM_PEOPLE):
    for ch in range(NUM_CHANNELS):
        channel_data = list()
        
        AE_ch = list()
        PE_ch = list()
        SE_ch = list()
        FE_ch = list()
        
        for epoch in range(NUM_EPOCHS):
            AE_item = AE_normal_matrix[person][epoch][ch][0]
            PE_item = PE_normal_matrix[person][epoch][ch][0]
            SE_item = SE_normal_matrix[person][epoch][ch][0]
            FE_item = FE_normal_matrix[person][epoch][ch][0]
            
            AE_ch.append(AE_item)
            PE_ch.append(PE_item)
            SE_ch.append(SE_item)
            FE_ch.append(FE_item)
            
            AE_item = AE_fatigue_matrix[person][epoch][ch][0]
            PE_item = PE_fatigue_matrix[person][epoch][ch][0]
            SE_item = SE_fatigue_matrix[person][epoch][ch][0]
            FE_item = FE_fatigue_matrix[person][epoch][ch][0]
            
            AE_ch.append(AE_item)
            PE_ch.append(PE_item)
            SE_ch.append(SE_item)
            FE_ch.append(FE_item)
            
        X_ALL["AE"].append(AE_ch)
        #y_ALL["AE"].append(create_y())
        X_ALL["PE"].append(PE_ch)
        #y_ALL["PE"].append(create_y())
        X_ALL["SE"].append(SE_ch)
        #y_ALL["SE"].append(create_y())
        X_ALL["FE"].append(FE_ch)
        #y_ALL["FE"].append(create_y())

    break

In [41]:
def create_y(size=NUM_EPOCHS):
    normal = [0 for i in range(size)]
    fatigue = [1 for i in range(size)]
    return normal + fatigue

In [70]:
# iterate channels
for entropy in X_ALL:
    print(entropy)
    for X in X_ALL[entropy]:
        X = np.array(X)
        np.nan_to_num(X, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
        
        # scale
        scaler = MinMaxScaler((-1, 1))
        X_scaled = scaler.fit_transform(X.reshape(-1, 1))

        # train test split
        y = create_y()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

        # train SVM
        svc = SVC(C=1000, gamma=0.001, kernel="rbf")
        model = svc.fit(X_train.reshape(-1, 1), y_train)

        h = model.predict(X_test.reshape(-1, 1))
        score = accuracy_score(y_true=y_test, y_pred=h)
        print(score)

AE
0.4866666666666667
0.48333333333333334
0.6633333333333333
0.48333333333333334
0.48333333333333334
0.48333333333333334
0.48333333333333334
0.48333333333333334
0.5333333333333333
0.48333333333333334
0.6166666666666667
0.48333333333333334
0.48333333333333334
0.48
0.48333333333333334
0.48
0.48333333333333334
0.48
0.4866666666666667
0.69
0.5233333333333333
0.48333333333333334
0.48333333333333334
0.48333333333333334
0.48
0.4866666666666667
0.48
0.48333333333333334
0.48
0.48333333333333334
PE
0.5033333333333333
0.4766666666666667
0.5733333333333334
0.4766666666666667
0.48333333333333334
0.5866666666666667
0.48333333333333334
0.5433333333333333
0.5166666666666667
0.4766666666666667
0.51
0.4766666666666667
0.55
0.48333333333333334
0.5033333333333333
0.48333333333333334
0.51
0.48333333333333334
0.48333333333333334
0.7033333333333334
0.5166666666666667
0.4766666666666667
0.58
0.48333333333333334
0.48333333333333334
0.48333333333333334
0.48333333333333334
0.4766666666666667
0.5
0.57
SE
0.483333